In [3]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing, ensemble, metrics
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

%matplotlib inline

In [7]:
train_dat = pd.read_csv('train_parsed.csv')
test_dat = pd.read_csv('test_parsed.csv')
train_labels = np.load('train_labels_parsed.npy')

In [16]:
dev_cutoff = int( len(train_labels) * 0.999 )
X_dev = train_dat[:dev_cutoff]
Y_dev = train_labels[:dev_cutoff]
X_test = train_dat[dev_cutoff:]
Y_test = train_labels[dev_cutoff:]

In [ ]:
n_folds = 5

skf = list(StratifiedKFold(Y_dev, n_folds))

clfs = [ xgb.XGBClassifier(n_estimators=100,objective='binary:logistic', max_depth=6,
                           colsample_bytree=0.75, subsample=0.83),
         xgb.XGBClassifier(n_estimators=100,objective='binary:logistic', max_depth=8,
                           colsample_bytree=0.75, subsample=0.83),
         xgb.XGBClassifier(n_estimators=100,objective='binary:logistic', max_depth=12,
                           colsample_bytree=0.75, subsample=0.83),
         #ensemble.RandomForestClassifier(n_estimators=20, criterion='gini'),
         #ensemble.ExtraTreesClassifier(n_estimators=40, criterion='gini')
    ]

-

In [ ]:
pred1 = clfs[0].predict(test_dat)
pred2 = clfs[1].predict(test_dat)
pred3 = clfs[2].predict(test_dat)
print(pred1)
pred = bclf.predict(np.concatenate((pred1,pred2,pred3)).reshape(len(pred1),3))

In [ ]:
#pred = bclf.predict(test_dat)
sample = pd.read_csv('sample_submission.csv')
sample.QuoteConversion_Flag = pred
sample.to_csv('xgb_stacked.csv', index=False)

# Blend a couple of models together

In [ ]:
n_folds = 5

skf = list(StratifiedKFold(Y_dev, n_folds))

depths = [6]

blend_train = np.zeros((X_dev.shape[0], len(depths)))
blend_test = np.zeros((X_test.shape[0], len(depths)))

for j, depth in enumerate(depths):
    print('Training classifier [%s]' % (j))
    blend_test_j = np.zeros((X_test.shape[0], len(skf))) # Number of testing data x Number of folds , we will take the mean of the predictions later
    for i, (train_index, cv_index) in enumerate(skf):
        print('Fold [%s]' % (i))
        
        param = {   'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'nthread':4,
            'max_depth':depth,
            'colsample_bytree':0.75,
            'subsample':0.83,
            'eta':0.025,
        }

        # This is the training and validation set    
        xg_train = xgb.DMatrix( X_dev.iloc[train_index].as_matrix(), label=Y_dev[train_index])
        xg_cv = xgb.DMatrix( Y_dev.iloc[cv_index].as_matrix(), label=train_labels[cv_index])

        m = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds=150)

        # This output will be the basis for our blended classifier to train against,
        # which is also the output of our classifiers
        blend_train[cv_index, j] = m.predict(X_cv)
        blend_test_j[:, i] = m.predict(X_test)
        # Take the mean of the predictions of the cross validation set
        blend_test[:, j] = blend_test_j.mean(1)
    
# Take the mean of the predictions of the cross validation set
blend_test_mean = blend_test.mean(1)
blend_pred_mean = blend_pred.mean(1)

# Get accuracy of the test


In [ ]:
t = pd.DataFrame(blend_test)
t['same'] = t.apply(lambda x:not ( x[0] == x[1] == x[2] == x[3] == x[4] ), axis=1 )
n = t[t['same'] == True].count()['same']
print(n, 'of', len(t), '(', n/len(t), ') are not in agreement')
print(blend_test_mean.shape, Y_test.shape)
acc = metrics.accuracy_score(blend_test_mean.round().astype('int'), Y_test.astype('int'))
print(acc)


In [ ]:
sample = pd.read_csv('sample_submission.csv')
sample.QuoteConversion_Flag = blend_pred_mean.round()
sample.to_csv('xgb_kfold.csv', index=False)